# BÀI TẬP PYTHON CƠ BẢN

## Đề Bài

Viết hàm tính tiền bảo hiểm và thuế thu nhập cá nhân cho các nhân viên:
- Input: lương, phụ cấp
- Output: BH, PIT

## Script

### Using

In [1]:
import sys
from enum import Enum
from collections import defaultdict

### Extension Methods

In [2]:
# Make sure input is integer
def inpInt():
    turn = 0
    while True:
        try:
            if turn > 0:
                print('Dữ liệu nhập vào phải là số nguyên, xin vui lòng thử lại: ', end='')
            val = input()
            num = int(val)
            break
        except:
            turn += 1
            continue
    return num

In [3]:
# Make sure input is float
def inpFloat():
    turn = 0
    while True:
        try:
            if turn > 0:
                print('Dữ liệu nhập vào phải là số, xin vui lòng thử lại: ', end='')
            val = input()
            num = float(val)
            break
        except:
            turn += 1
            continue
    return num

In [4]:
# Input integer large than 0
def inpIntPlus():
    turn = 0
    while True:
        if turn > 0:
            print('Dữ liệu nhập vào phải là số nguyên dương, xin vui lòng thử lại: ', end='')
        num = inpInt()
        if num > 0:
            return num
        turn += 1

In [5]:
# Input float without minus
def inpFloatWoMs():
    turn = 0
    while True:
        if turn > 0:
            print('Dữ liệu nhập vào phải là số không âm, xin vui lòng thử lại: ', end='')
        num = inpFloat()
        if num >= 0:
            return num
        turn += 1

In [6]:
# Input integer plus with caption
def inpIntPlusWithCap(cap='Nhập: '):
    print(cap, end='')
    text = inpIntPlus()
    return text

In [7]:
# Input float plus with caption
def inpFloatPlusWithCap(cap='Nhập: '):
    print(cap, end='')
    text = inpFloatWoMs()
    return text

### Contract Fields

In [8]:
l_cs = 1800000  # lương cơ sở 2023 (LCS)
l_ttv = 4680000  # luong tối thiểu vùng 2023 (LTTV)

In [9]:
pt_bhxh = 8  # phần trăm bảo hiểm xã hội (BHXH)
pt_bhyt = 1.5  # phần trăm bảo hiểm y tế (BHYT)
pt_bhtn = 1  # phần trăm bảo hiểm thất nghiệp (BHTN)

In [10]:
hs_bh_max = 20  # hệ số bảo hiểm (BH) tối đa

In [11]:
gtt_cn = 11000000  # giảm trừ thuế thu nhập cá nhân (TTNCN)

In [12]:
class BacThue(Enum):
    Bac0 = 0
    Bac1 = 1
    Bac2 = 2
    Bac3 = 3
    Bac4 = 4
    Bac5 = 5
    Bac6 = 6
    Bac7 = 7

In [13]:
class NhanVien:
    # Initialization
    def __init__(self, lcb=0, pc=0):
        self.luongcoban = lcb
        self.phucap = pc

    # Set lương thực nhận
    def setltn(self, ltn=0):
        self.luongthucnhan = ltn

In [14]:
i_bt_src = 0  # index bậc thuế source
i_bt_dst = 1  # index bậc thuế destination
i_bt_rslt = 2  # index bậc thuế result

# lương từ | đến dưới | bậc thuế
bt = [[0, 5000000, BacThue.Bac1],
      [5000000, 10000000, BacThue.Bac2],
      [10000000, 18000000, BacThue.Bac3],
      [18000000, 32000000, BacThue.Bac4],
      [32000000, 52000000, BacThue.Bac5],
      [52000000, 80000000, BacThue.Bac6],
      [80000000, sys.maxsize, BacThue.Bac7]]

In [15]:
i_bt_ts = 0  # index thuế suất của từng bậc thuế
i_bt_hsg = 1  # index hệ số giảm của từng bậc thuế

bt_dics = defaultdict(list)
# {bậc thuế} - {thuế suất (%) | hệ số giảm}
bt_dics[BacThue.Bac0] = [0, 0]
bt_dics[BacThue.Bac1] = [5, 0]
bt_dics[BacThue.Bac2] = [10, 250000]
bt_dics[BacThue.Bac3] = [15, 750000]
bt_dics[BacThue.Bac4] = [20, 1650000]
bt_dics[BacThue.Bac5] = [25, 3250000]
bt_dics[BacThue.Bac6] = [30, 5850000]
bt_dics[BacThue.Bac7] = [35, 9850000]

### Contract Methods

In [16]:
# Bảo hiểm xã hội calculate
bhxhCalc = lambda lcb=0: min(lcb, l_cs * hs_bh_max) * pt_bhxh / 100  # mức lương đóng BHXH tối đa bằng 20 lần mức LCS

In [17]:
# Bảo hiểm y tế calculate
bhytCalc = lambda lcb=0: min(lcb, l_cs * hs_bh_max) * pt_bhyt / 100 # mức lương đóng BHYT tối đa bằng 20 lần mức LCS

In [18]:
# Bảo hiểm thất nghiệp calculate
bhtnCalc = lambda lcb=0: min(lcb, l_ttv * hs_bh_max) * pt_bhtn / 100  # mức lương đóng BHTN tối đa bằng 20 lần mức LTTV

In [19]:
# Bảo hiểm calculate
bhCalc = lambda lcb=0: bhxhCalc(lcb) + bhytCalc(lcb) + bhtnCalc(lcb)  # gói đóng BH gồm có BHXH, BHYT & BHTN

In [20]:
# Lương NET calculate
lNetCalc = lambda lcb=0: lcb - bhCalc(lcb)  # lương NET là lương cơ bản (LCB) sau khi đóng BH

In [21]:
# Lương thực tế calculate
lttCalc = lambda lcb=0, pc=0: lNetCalc(lcb) + pc  # lương thực tế là lương NET được tính thêm các khoản phụ cấp

In [22]:
# Bậc thuế scan
def btScan(ltt=0):
    if ltt > gtt_cn:
        tntt = ltt - gtt_cn  # thu nhập tính thuế (TNTT) là số dư của lương thực tế (LTT) so với mức giảm trừ TTNCN
        for i in range(len(bt)):
            bacThue = bt[i]
            if bacThue[i_bt_src] <= tntt < bacThue[i_bt_dst]:
                return bacThue[i_bt_rslt]
    else:
        return BacThue.Bac0

In [23]:
# Thuế thu nhập cá nhân calculate
def ttncnCalc(ltt=0):
    btDic = bt_dics[btScan(ltt)]
    return (ltt - gtt_cn) * btDic[i_bt_ts] / 100 - btDic[i_bt_hsg]  # TTNCN chỉ đánh trên (TNTT)

In [24]:
# Lương thực nhận calculate
ltnCalc = lambda ltt=0: ltt - ttncnCalc(ltt)  # lương sau thuế (LST) bằng lương thực nhận (LTN)

In [30]:
# Main
def main():
    n = inpIntPlusWithCap('Nhập số lượng nhân viên: ')
    nhanViens = []
    for i in range(n):
        lcb = inpFloatPlusWithCap(f'Nhập lương cơ bản cho nhân viên {i + 1}: ')
        pc = inpFloatPlusWithCap(f'Nhập phụ cấp cho nhân viên {i + 1}: ')
        nhanVien = NhanVien(lcb, pc)
        nhanViens.append(nhanVien)
        nhanVien.setltn(ltnCalc(lttCalc(lcb, pc)))
    print('\nKết quả')
    for i in range(len(nhanViens)):
        print(f'Lương thực nhận của nhân viên {i + 1} là: {"{:,}".format(nhanViens[i].luongthucnhan)}')

In [33]:
# Run
main()

Nhập số lượng nhân viên: a
Dữ liệu nhập vào phải là số nguyên, xin vui lòng thử lại: 1.2
Dữ liệu nhập vào phải là số nguyên, xin vui lòng thử lại: -3
Dữ liệu nhập vào phải là số nguyên dương, xin vui lòng thử lại: 0
Dữ liệu nhập vào phải là số nguyên dương, xin vui lòng thử lại: 4
Nhập lương cơ bản cho nhân viên 1: b
Dữ liệu nhập vào phải là số, xin vui lòng thử lại: -5
Dữ liệu nhập vào phải là số không âm, xin vui lòng thử lại: 0
Nhập phụ cấp cho nhân viên 1: c
Dữ liệu nhập vào phải là số, xin vui lòng thử lại: -6
Dữ liệu nhập vào phải là số không âm, xin vui lòng thử lại: 44444444.4
Nhập lương cơ bản cho nhân viên 2: 88888888
Nhập phụ cấp cho nhân viên 2: 0
Nhập lương cơ bản cho nhân viên 3: 5000000
Nhập phụ cấp cho nhân viên 3: 17000000
Nhập lương cơ bản cho nhân viên 4: 20000000
Nhập phụ cấp cho nhân viên 4: 2000000

Kết quả
Lương thực nhận của nhân viên 1 là: 39,333,333.3
Lương thực nhận của nhân viên 2 là: 68,355,999.384
Lương thực nhận của nhân viên 3 là: 20,653,750.0
Lương thực